In [5]:
#%load_ext autoreload
%autoreload 2

import sqlite3
import psycopg2
import pandas as pd
import numpy as np
import math
import io
from DataBase_functions import *
from DataBase_functions_testing import *

#Google sheed API
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

<p style="font-size: 50px; text-align: center; color: #20a7e5;">Creating definitive data-frame</p>

<p style="font-size: 25px; color: #208ee5">Registro actividad todos</p>

In [6]:
# Email service account that need to share the google sheet: matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com" 

#Permisos que solicitará a la cuenta de google
scope = [
    'https://spreadsheets.google.com/feeds',        #Scope antiguo de Sheet API (v3)
    'https://www.googleapis.com/auth/spreadsheets', #Scope moderno de Sheet API (v4)
    'https://www.googleapis.com/auth/drive'         #Permisos para Google Drive
    ]

credentials = "credentials.json" #Credenciales del proyecto de Google Cloud

headers_definitive = ["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios", "version_date", "version", "petition_arq", "dev_master"]
try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    spreadsheet_id = ["1s5XhPukFbAsQmWMBDkkoSO5l_M8Muuo-1CrNC0LC-Yw", "1mUJAhezlVOj2TVWBs3loo8C91aDWzqaYjboDysidpO0"] #[2025, 2024]  

    #Dataframe con los datos de todos los meses
    headers = ["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]
    df_rat = pd.DataFrame(columns=headers)
    for i in spreadsheet_id: 
        # Open the Google Sheet file by ID
        spreadsheet = client.open_by_key(i)

        # List all sheets file
        sheets = spreadsheet.worksheets()
        print(f"Available sheeds: {[sheet.title for sheet in sheets]}")
 
        meses = ["2025", "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre", "Enero_25"]
        for sheet in sheets:
            sheet_name = sheet.title
            print(sheet_name) 
            sheet.update(values=[headers], range_name='A1')
            if sheet_name in meses:
                df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=headers))
                df_rat = pd.concat([df_rat, df_aux], ignore_index=True)
            else: 
                print(sheet_name, "is not a month")
                pass
    #df = df.drop_duplicates(subset='Código', keep='first')
    df_rat = df_rat[headers] #Únicamente me quedo con las columnas que me interesa
    print(df_rat.shape, "\n", df_rat.columns)

    #Dataframe validation
    df_rat.rename(columns={"Validada": "validada", "Responsable": "responsable", "Gestor DDBB": "DDBB", "Geografía": "geography", "Petición": "petition_code", "Feature": "feature", "SDATOOL": "sdatool", "Código": "DQDP_code", "Gestor BBDD": "DDBB", "Horas": "duration_time", "Comentarios": "description", "Fecha de alta": "fecha_in", "Fecha de fin": "fecha_out"}, inplace=True)

    df_rat["fecha_in"] = df_rat["fecha_in"].astype(str)
    df_rat["Fecha incurrida"] = df_rat["Fecha incurrida"].astype(str)
    df_rat["fecha_out"] = df_rat["fecha_out"].astype(str)
    df_rat["UUAA"] = df_rat["UUAA"].astype(str)
    df_rat["DQDP_code"] = df_rat["DQDP_code"].astype(str)
    df_rat["sdatool"] = df_rat["sdatool"].astype(str)
    df_rat["feature"] = df_rat["feature"].astype(str)
    df_rat["geography"] = df_rat["geography"].astype(str)
    df_rat["DDBB"] = df_rat["DDBB"].astype(str)
    df_rat["Ámbito"] = df_rat["Ámbito"].astype(str)
    df_rat["responsable"] = df_rat["responsable"].astype(str)
    df_rat["validada"] = df_rat["validada"].astype(str)
    df_rat["duration_time"] = pd.to_numeric(df_rat["duration_time"], errors="coerce")
    df_rat["description"] = df_rat["description"].astype(str)
    df_rat["petition_code"] = df_rat["petition_code"].astype(str)

    df_rat["version_date"] = "Nan"
    df_rat["version"] = "Nan"
    df_rat["petition_arq"] = "Nan"
    df_rat["dev_master"] = "Nan"

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

print(df_rat, "\n", df_rat.columns)

Available sheeds: ['2025 totales - Peticiones', '2025 totales - Peticiones - Pro', '2025', 'Gráficos Semanales - Procesos', '2025 - Procesos', 'Enero_25', 'Responsables', 'Copia de 2025']
2025 totales - Peticiones
2025 totales - Peticiones is not a month
2025 totales - Peticiones - Pro
2025 totales - Peticiones - Pro is not a month
2025
Gráficos Semanales - Procesos
Gráficos Semanales - Procesos is not a month
2025 - Procesos
2025 - Procesos is not a month
Enero_25
Responsables
Responsables is not a month
Copia de 2025
Copia de 2025 is not a month
Available sheeds: ['2024 totales - Peticiones', '2024 totales - Peticiones - Pro', '2024', 'Gráficos Semanales - Procesos', '2024 - Procesos', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Enero_25', 'Responsables', 'None', 'Peticiones totales', 'All petitions', 'General']
2024 totales - Peticiones
2024 totales - Peticiones is not a month
2024 totales - Peticiones - Pro
2024 totales - Peticio

<p style="font-size: 35px; color: #208ee5;">Peticiones globales Oracle, Elastic Search y Mongo DB</p>

<p style="font-size: 25px; color: #208cc5;;">Oracle Physics, Elastic Search, Mongo DB</p>

In [7]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo. #Oracle Physics, Elastic Search, Mongo DB v1, Mongo DB v2 
    spreadsheet_id = [
        "18_nIbDi3DBd4pMJb_yUoxvyOShj4x1WCWar4LjD6m70", # Oracle Physics
        "19h-4hJzJvN0b5n6VZRNAQITs5xBEqZRZvG5TYCEm8Vo", # Elastic Search
        "1xZ7yFsQE429HqAXGUooHWbTxfaLB5tBDDG6nK_L6q2U", # Mongo DB v1
        "1eBua4aoheK-TCnXDNyScWwTNc226fsXXEgRlXpO6eG0"  # Mongo DB v2
    ] 

    #DataFrame con los datos del excel
    headers = ["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Path", "Comentarios"]
    df_globales = pd.DataFrame(columns=headers)
    for i in spreadsheet_id: 
        # Open the Google Sheet file by ID
        spreadsheet = client.open_by_key(i)

        # List all sheets file
        sheets = spreadsheet.worksheets()
        print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

        for sheet in sheets: 
            try: 
                print(sheet.title)
                sheet.update(values=[headers], range_name='A1') #Nombrando las columnas

                df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=headers))
                df_globales = pd.concat([df_globales, df_aux], ignore_index=True)
                print(df_globales.columns)
                #Poniendo el DDBB correspondiente
                if i == spreadsheet_id[0]: 
                    df_globales["DDBB"] = "Oracle Physics"
                elif i == spreadsheet_id[1]: 
                    df_globales.loc[df_globales["DDBB"] != "Oracle Physics", "DDBB"] = "Elastic Search"
                elif i == spreadsheet_id[2] or i == spreadsheet_id[3]: 
                    df_globales.loc[(df_globales["DDBB"] != "Oracle Physics") & (df_globales["DDBB"] != "Elastic Search"), "DDBB"] = "Mongo DB"
                else: 
                    print("Fuera de rango spreadsheet_id")

            except gspread.exceptions.APIError as e:
                if e.response.status_code == 429: 
                    wait_time = 6
                    print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else: 
                    pass
            
    #Creating DDBB and geography column
    df_globales["geography"] = "Global"

    print(df_globales.shape)

    df_globales["UUAA"] = df_globales["UUAA"].astype(str)
    df_globales["Petición Jira"] = df_globales["Petición Jira"].astype(str)
    df_globales["Versión Repo"] = df_globales["Versión Repo"].astype(str)
    df_globales["Petición ARQ"] = df_globales["Petición ARQ"].astype(str)
    df_globales["Fecha cierre"] = df_globales["Fecha cierre"].astype(str)
    df_globales["Comentarios"] = df_globales["Comentarios"].astype(str)
    df_globales["Path"] = df_globales["Path"].astype(str)
    df_globales["DDBB"] = df_globales["DDBB"].astype(str)
    df_globales["geography"] = df_globales["geography"].astype(str)

    df_globales.rename(columns={"Path": "path", "Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)
    df_globales = df_globales[["petition_code", "version", "petition_arq", "fecha_out", "description", "path", "UUAA", "DDBB", "geography"]]
except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

print(df_globales.columns, df_globales.shape)

Available sheeds: ['APIC', 'ATAU', 'B1DS', 'B43T', 'BJP9', 'CCUS', 'DAMS', 'DMTO', 'EGRP', 'EI4M', 'ESCD', 'FCRK', 'FZHI', 'GDEL', 'GHDC', 'GJSX', 'GSIN', 'GUAR', 'H9IY', 'HFPP', 'HLCY', 'HPGR', 'IMPT', 'J6G7', 'K5VI', 'KAPI', 'KARC', 'KBGE', 'KBTQ', 'KCCX', 'KCDO', 'KCMC', 'KCMS', 'KCNC', 'KCNS', 'KCOG', 'KCOL', 'KCSN', 'KCSP', 'KDCL', 'KDFH', 'KDMG', 'KECF', 'KERF', 'KESJ', 'KFGC', 'KFRE', 'KFUL', 'KGCC', 'KGCI', 'KGTP', 'KGWY', 'KIFV', 'KIVS', 'KLIM', 'KLNE', 'KMET', 'KMIC', 'KMOL', 'KMRK', 'KMYC', 'KNWD', 'KOMN', 'KONB', 'KPAG', 'KPAD', 'KPAY', 'KPDA', 'KPDR', 'KPFE', 'KPFM', 'KPIG', 'KPPS', 'KPST', 'KRBA', 'KRCE', 'KSAG', 'KSAN', 'KSDD', 'KSPH', 'KSRO', 'KUSU', 'KVCS', 'KXSG', 'KYSZ', 'KYOP', 'KYUB', 'MCRR', 'MFCG ', 'MK0W', 'MDDM', 'NI0C', 'O43T', 'PGVC', 'PIVU', 'PMSC', 'PMSV', 'P6ES', 'QWJR', 'QWPJ', 'RLR7', 'SWP3', 'S47B', 'UGX2', 'V34F', 'VAMO', 'W1BD', 'XBOB', 'ZGR7', 'XMBR', 'XR6Z', 'ZDNM', 'ZNXD']
APIC
Index(['UUAA', 'Petición Jira', 'Versión Repo', 'Petición ARQ', 'Fecha 

<p style="font-size: 35px; color: #208ee5;">dqdp_portal CSV</p>

In [8]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

service = build('drive', 'v3', credentials=creds)

#Drive, folder id
folder_id = '1w2V9GaUCXD4vdjolQjIAEMujMOiqYwEP'

#list of archives
results = service.files().list(
    q = f"'{folder_id}' in parents", 
    pageSize=120, 
    fields = "nextPageToken, files(id, name)"
).execute()
files = results.get('files', [])

#List that can take all dataframe from csv files
dfs = []

# Itera sobre los archivos y lee los CSV
for file in files:
    if file['name'].endswith('.csv'): # Solo procesa archivos CSV
        file_id = file['id']
        # Descarga el archivo CSV a un archivo temporal
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO() # Usamos un archivo en memoria para no guardar el archivo
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()

        fh.seek(0) # Reiniciar el puntero al inicio del archivo
        try:
            df_aux = pd.read_csv(fh) # Lee el CSV con Pandas
            dfs.append(df_aux)
        except Exception as e:
            print(f"Error al leer el archivo {file['name']}: {e}")

# Combina todos los DataFrames en uno solo
df_csv_combined = pd.concat(dfs, ignore_index=True)

In [9]:
df_csv_combined.rename(columns={'Código': 'DQDP_code', 'Responsable': 'responsable', 'Fecha de alta': 'fecha_in', 'Petición': 'petition_code', 'Geografía': 'geography', 'Horas esfuerzo': 'duration_time', 'Gestor': 'DDBB', 'Fecha fin': 'fecha_out'}, inplace=True)
df_csv_combined['description'] = df_csv_combined['Descripción corta'] + '\n' + df_csv_combined['Descripción']
df_csv_combined = df_csv_combined[['DQDP_code', 'UUAA', 'responsable', 'fecha_in', 'petition_code', 'geography', 'duration_time', 'DDBB', 'description']]

df_csv_combined.columns

Index(['DQDP_code', 'UUAA', 'responsable', 'fecha_in', 'petition_code',
       'geography', 'duration_time', 'DDBB', 'description'],
      dtype='object')

<p style="font-size: 35px; color: #20cff5; text-align: center;">All in one Data Frame</p>

In [10]:
#Creating global data-frame
df_combined = pd.DataFrame(columns=list(df_rat.columns))
df_combined = pd.concat([df_combined, df_rat, df_globales, df_csv_combined], ignore_index=True)
print(df_combined.columns)


columns_to_use = ["fecha_in", "fecha_out", "UUAA", "DQDP_code", "sdatool", "feature", "geography", "DDBB", "Ámbito", "responsable", "validada", "description", "version_date", "version", "petition_arq", "petition_code", "duration_time", "dev_master"]
for column in columns_to_use: 
    df_combined[column] = df_combined[column].astype(str)

df_combined["duration_time"] = pd.to_numeric(df_combined["duration_time"], errors="coerce")
df_combined["dev_master"] = "None"

#Poniendo "None" o np.nan donde corresponde


for row in df_combined.iterrows(): 
    for key, value in dict(row[1]).items():
        if value == "Nan" or value == None or value == "" or value == "nan" or value == "None" or value == "none" or value == "NONE" or value == np.nan: 
            if key == "fecha_in" or key == "fecha_out" or key == "Fecha incurrida" or key == "UUAA" or key == "DQDP_code" or key == "sdatool" or key == "feature" or key == "geography" or key == "DDBB" or key == "Ámbito" or key == "responsable" or key == "validada" or key == "description" or key == "version_date" or key == "version" or key == "petition_arq" or key == "petition_code" or key == "dev_master":
                df_combined.at[row[0], key] = "None"
            if key == "duration_time":
                df_combined.at[row[0], key] = np.nan
            else: 
                pass



df_combined = df_combined[df_combined["petition_code"] != "None"]

agg_dict = {
    col: 'first' for col in df_combined.columns if col not in ["description", "duration_time"]
}
agg_dict['description'] = lambda x: "\n".join(x.astype(str))
agg_dict["duration_time"] = 'sum'

#Data frame final a partir del campo 'poetition_code'
#df_g.drop_duplicates(subset=["petition_code"], keep="first", inplace=True) #Método 1
final_df = df_combined.groupby("petition_code", as_index=False).agg(agg_dict) #Método 2

/var/folders/r3/_0ct4tcx0jj4q9jfbhwjb93m0000gn/T/ipykernel_6170/2962138477.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_combined, df_rat, df_globales, df_csv_combined], ignore_index=True)


Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'duration_time', 'description',
       'version_date', 'version', 'petition_arq', 'dev_master', 'path'],
      dtype='object')


In [12]:
petitions = set(final_df["petition_code"])
print(len(petitions), final_df.columns)
petitions

6120 Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'version_date', 'version', 'petition_arq',
       'dev_master', 'path', 'description', 'duration_time'],
      dtype='object')


{'REQ000100965592',
 'OTRSSD-18090',
 'OTRSSD-11951',
 'OTRSSD-4368',
 'OTRSSD-5337',
 'OTRSSD-14200',
 'OTRSSD-17265',
 'TPIA_20211111100422',
 'SGS-19553',
 'OTRSSD-18885',
 'OTRSSD-16189',
 'OTRSSD-16663',
 'SGS-13501',
 'OTRSSD-7996',
 'SGS-16766',
 'SGS-17743',
 'WO0000101379563',
 'KDHJ_20240426120049',
 'OTRSSD-10360',
 'OTRSSD-7464',
 'OTRSSD-7336',
 'OTRSSD-17517',
 'DSFI_20221013162141',
 'SGS-11310',
 'SGS-25492',
 'OTRSSD-10978',
 'FSIF_20211029180352',
 'REQ000012257959',
 'OTRSSD-17340',
 'REQ000101122077',
 'OTRSSD-18944',
 'KPNA_20210519154745',
 'OTRSSD-10192',
 'OTRSSD-11245',
 'REQ000012020294',
 'KGPE_20211223140421',
 'SMIA_20210621081438',
 'SGS-16084',
 'SGS-20202',
 'WO0000101284883',
 'SGS-19748',
 'REQ000101038198',
 'KGPE_20211013120418',
 'OTRSSD-17574',
 'OTRSSD-13238',
 'SGS-16103',
 'SGS-14124',
 'SGS-20026',
 'REQ000101202670',
 'REQ000100553615',
 'SGS-14984',
 'OTRSSD-14169',
 'OTRSSD-17362',
 'OTRSSD-3466',
 'OTRSSD-16994',
 'SGS-10486',
 'SGS-7844',


In [13]:
#Poniendo "None" o None donde corresponda
for petition in petitions: 
    dic_aux = {i: [] for i in columns_to_use}

    for column in columns_to_use: 
        df_filtered = final_df[final_df["petition_code"] == petition]
        
        if not df_filtered.empty: 
            df_value = df_filtered[column].values[0]
        else: 
            df_value = "None" if column != "duration_time" else None
        
        if df_value != "None" and df_value is not None: 
            dic_aux[column].append(df_value)
        else:
            dic_aux[column].append(df_value) #None o "None"

    df_aux = pd.DataFrame(dic_aux)
    final_df = pd.concat([final_df, df_aux], ignore_index=True)

agg_dict = {
col: 'first' for col in final_df.columns if col not in ["description", "duration_time"]
}
agg_dict['description'] = lambda x: "\n".join(x.astype(str))
agg_dict["duration_time"] = 'sum'
final_df = final_df.groupby("petition_code", as_index=False).agg(agg_dict)

In [14]:
#Contar los valores "None" o None
df_none_count = {i: 0 for i in list(final_df.columns)}
for row in final_df.iterrows(): 
    for key, value in dict(row[1]).items():
        if value == "None" or value == None or (isinstance(value, float) and np.isnan(value)):
            df_none_count[key] += 1
        else: 
            pass
print(final_df.shape, final_df.shape, final_df.shape)
df_none_count


(6120, 20) (6120, 20) (6120, 20)


{'fecha_in': 1061,
 'Fecha incurrida': 4749,
 'fecha_out': 4306,
 'UUAA': 1,
 'DQDP_code': 812,
 'sdatool': 4727,
 'feature': 4753,
 'petition_code': 0,
 'geography': 265,
 'DDBB': 181,
 'Ámbito': 4857,
 'responsable': 1067,
 'validada': 5246,
 'version_date': 6120,
 'version': 5322,
 'petition_arq': 5341,
 'dev_master': 6120,
 'path': 4904,
 'description': 0,
 'duration_time': 0}

---
---

<p style="font-size: 45px; text-align: center; color: green;">Cleaning definitive dataframe</p>

In [20]:
print(final_df["geography"].unique())
print(final_df["DDBB"].unique())
#final_df.drop(columns=['', 'path'], inplace=True)
print(final_df.columns)

['Global' 'España' 'None' 'CIB' 'Holding' 'México' 'Perú' 'Colombia'
 'Argentina']
['Oracle Physics' 'DB2 Host' 'Elastic Search' 'DB2 UDB' 'None' 'Oracle R2'
 'Teradata' 'Mongo DB' 'PostgreSQL' 'OTRSSD-16335' 'ElasticSearch' 'Mongo'
 'Netezza' 'DB2' 'MySQL server' 'HDFS Parquet' 'HDFS Avro']
Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'version_date', 'version', 'petition_arq',
       'dev_master', 'path', 'description', 'duration_time'],
      dtype='object')


In [19]:
final_df["DDBB"] = final_df["DDBB"].replace(["DB2 HOST"], "DB2 Host")
final_df["DDBB"] = final_df["DDBB"].replace(["Oracle Physics", "ORACLE Physics"], "Oracle Physics")
final_df["DDBB"] = final_df["DDBB"].replace(["ELASTICSEARCH", "ElasTICSEARCH", "ElaSTICSEARCH"], "Elastic Search")
final_df["DDBB"] = final_df["DDBB"].replace(["ORACLE R2", "oracle r2"], "Oracle R2")
final_df["DDBB"] = final_df["DDBB"].replace(["teradata", "TERADATA"], "Teradata")
final_df["DDBB"] = final_df["DDBB"].replace(["MongoDB", "MONGO DB", "MongoDB", "MONGODB", "Mongo\t"], "Mongo DB")
final_df["DDBB"] = final_df["DDBB"].replace(["POSTGRESS R2", "POSTGRESS Physics", "PosgreSQL"], "PostgreSQL")
final_df["DDBB"] = final_df["DDBB"].replace(["NETEZZA"], "Netezza")

final_df["geography"] = final_df["geography"].replace(["España/CIB", "España-CIB"], "CIB")
final_df["geography"] = final_df["geography"].replace(["España "], "España")

In [21]:
final_df["fecha_in"] = final_df["fecha_in"].astype(str)
final_df["Fecha incurrida"] = final_df["Fecha incurrida"].astype(str)
final_df["fecha_out"] = final_df["fecha_out"].astype(str)
final_df["UUAA"] = final_df["UUAA"].astype(str)
final_df["DQDP_code"] = final_df["DQDP_code"].astype(str)
final_df["sdatool"] = final_df["sdatool"].astype(str)
final_df["feature"] = final_df["feature"].astype(str)
final_df["petition_code"] = final_df["petition_code"].astype(str)
final_df["geography"] = final_df["geography"].astype(str)
final_df["DDBB"] = final_df["DDBB"].astype(str)
final_df["responsable"] = final_df["responsable"].astype(str)
final_df["version"] = final_df["version"].astype(str)
final_df["petition_arq"] = final_df["petition_arq"].astype(str)
final_df["dev_master"] = final_df["dev_master"].astype(str)
final_df["path"] = final_df["dev_master"].astype(str)
final_df["description"] = final_df["description"].astype(str)
final_df["duration_time"] = pd.to_numeric(final_df["duration_time"], errors="coerce")

---

<p style="font-size: 25px; color: #00d17f;">Actualizar y guardar</p>

In [22]:
#Actualizar y guardar

# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

#Cargar credenciales
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

#Autorizar gspread con las credenciales
client = gspread.authorize(creds)

# ID de la hoja de cálculo 
srpeadsheet_id = "1biaKHw0fV5w5HBWsAto2Q-WCLkXe4bwc72hoS6BNDe8" #petitions file

# Open the Google Sheet file by ID
spreadsheet = client.open_by_key(srpeadsheet_id)

# Select worksheet
worksheet = spreadsheet.worksheet('All petitions')

set_with_dataframe(worksheet, final_df, include_index=False, include_column_header=True, resize=True)

###############################################################################################################
#Guardo el df en un excel en la carpeta actual
final_df.to_excel("petitions.xlsx", index=False, sheet_name="All petitions")

In [23]:
final_df

,fecha_in,Fecha incurrida,fecha_out,UUAA,DQDP_code,sdatool,feature,petition_code,geography,DDBB,Ámbito,responsable,validada,version_date,version,petition_arq,dev_master,path,description,duration_time
0,None,None,13/09/2023,KMYC,None,None,None,,Global,Oracle Physics,None,None,None,None,23,DATASD-117636,None,None,cambiar el nombre de un índice\nNone\n camb...,0.0
1,23/10/2024,11/11/2024,11/11/2024,KDHN,DQ-DP-007223,47764,ADMMAYOR-136,KDHN_20241022115246,España,DB2 Host,HOST,Pedro,None,None,None,None,None,None,ampliar un campo\nampliar un campo\nampliar un...,12.0
2,None,None,03/02/2023,KTRA,None,None,None,OTRSSD-5682,Global,Elastic Search,None,None,None,None,16,DATASD-62338,None,None,None\nNone,0.0
3,19/12/2022,None,None,TVPV,DQ-DP-003399,None,None,-TVPV_20221212130816,España,DB2 Host,None,RAMON GARCIA IRANZO,None,None,None,None,None,None,None\nNone,0.0
4,28/02/2023,None,None,TVPV,DQ-DP-003823,None,None,A2027920,España,DB2 Host,None,JUAN ALFONSO BARRIOS PATIÑO,None,None,None,None,None,None,None\nNone,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6115,12/02/2025,None,None,PMSV,12,Esperando,Info,WO0000101463364,Global,Oracle Physics,NextGEN,None,None,None,None,None,None,None,None\nNone\nNone\nNone,0.0
6116,14/02/2025,14/02/2025,14/02/2025,KCSP,DQ-DP-007783,44974,WSONEPIV18-4449,WO0000101468509,Global,Oracle Physics,NextGEN,Matías,None,None,None,None,None,None,3.1.3 master v13 --> 3.4.3 dev v2 (segundo int...,9.0
6117,14/02/2025,None,None,XMBR,None,44974,ADDC-333,WO0000101470693,Global,Mongo DB,Legacy,None,None,None,None,None,None,None,None\nNone,0.0
6118,26/11/2021,None,None,XTRE,DQ-DP-001079,None,None,XTRE_20211117092400,España,DB2 Host,None,ARANZAZU MIRANDA POSTIGO,None,None,None,None,None,None,None\nNone,0.0


---
---

In [24]:
for row in final_df.iterrows():
    insert_data_testing(dict(row[1]))

<p style="font-size: 40px; color: grey;">Testing</p>

In [25]:
conn1 = sqlite3.connect("BBVA_testing.db")
cursor1 = conn1.cursor()
cursor1.execute("PRAGMA foreign_keys = ON") #In sqlite3 foreign keys are disabled by default

#######
cursor1.execute("SELECT UUAA FROM UUAA")
all_uuaa = [x[0] for x in cursor1.fetchall()]

#######
conn1.commit()
cursor1.close()
conn1.close()

In [ ]:
'''
conn3 =  psycopg2.connect(
        dbname = "pwd_control_plnk", 
        user = "matublaq",
        password = "SF19KOpSPMl8Ru51ONQ33AHOf0RuZnne", 
        host = "dpg-ctevf3t6l47c73b4jadg-a.oregon-postgres.render.com",
        port = "5432"
)   
cursor3 = conn3.cursor()

##########


##########
conn3.commit()
cursor3.close()
conn3.close()
'''

'\nconn3 =  psycopg2.connect(\n        dbname = "pwd_control_plnk", \n        user = "matublaq",\n        password = "SF19KOpSPMl8Ru51ONQ33AHOf0RuZnne", \n        host = "dpg-ctevf3t6l47c73b4jadg-a.oregon-postgres.render.com",\n        port = "5432"\n)   \ncursor3 = conn3.cursor()\n\n##########\n\n\n##########\nconn3.commit()\ncursor3.close()\nconn3.close()\n'

In [26]:
all_uuaa

['----------------',
 'AAQB',
 'AAQQ',
 'ABCD',
 'ABQB',
 'ABQN',
 'ACTK',
 'ADAS',
 'ADEX',
 'ADHR',
 'ADIF',
 'ADIT',
 'AEAT',
 'AFDE',
 'AGRE',
 'APIC',
 'AQMS',
 'ASAN',
 'ATAU',
 'B1DS',
 'B43T',
 'BBRE',
 'BDCT',
 'BDUE',
 'BDUS',
 'BJP9',
 'CACA',
 'CAHI',
 'CAIG',
 'CBIL',
 'CCMV',
 'CCRT',
 'CCTD',
 'CCUS',
 'CCYB',
 'CECE',
 'CECR',
 'CGCB',
 'CGCM',
 'CGMV',
 'CGSJ',
 'CGUG',
 'CHM8',
 'CKDZ',
 'CLOG',
 'CMFI',
 'CMOR',
 'COBO',
 'COPV',
 'CPAD',
 'CPAN',
 'CPCP',
 'CRDC',
 'CSTM',
 'CUOS',
 'CXCX',
 'CXOD',
 'DALU',
 'DAMS',
 'DMTO',
 'DSCZ',
 'DSDM',
 'DSFI',
 'DSFP',
 'DSGD',
 'DSOC',
 'DSPC',
 'DSPM',
 'DSRA',
 'DSRV',
 'EARI',
 'EBDP',
 'EBEX',
 'EBOU',
 'EBPR',
 'EBPS',
 'EBWR',
 'ECAR',
 'ECBK',
 'ECCC',
 'ECCF',
 'ECFM',
 'ED5J',
 'EDBP',
 'EDOM',
 'EDPF',
 'EECI',
 'EFRL',
 'EGAT',
 'EGFE',
 'EGPI',
 'EGRP',
 'EGUS',
 'EHRA',
 'EHRH',
 'EHTC',
 'EHUB',
 'EHUD',
 'EI4M',
 'EICF',
 'EIFV',
 'EIPI',
 'EJNF',
 'EJTM',
 'EKAS',
 'EKDL',
 'EKPL',
 'EKSA',
 'EKSI',
 'EKVL'